In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
import matplotlib.pyplot as plt
import seaborn as sns

def prepare_data(df, target_currency, sequence_length=10):
    """
    Підготовка даних для навчання Transformer-моделі
    та обчислення вагових коефіцієнтів (sample_weight).
    """
    data = df.values
    
    # Нормалізація
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    
    X, y = [], []
    date_array = df.index.values  # datetime64[ns]
    
    target_idx = df.columns.get_loc(target_currency)
    
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:(i + sequence_length), :])
        y.append(scaled_data[i + sequence_length, target_idx])
        
    X = np.array(X)
    y = np.array(y)
    
    # Формуємо вагові коефіцієнти (експоненціальне згасання впливу старих даних)
    sample_weights = []
    last_date = date_array[-1]
    alpha = 0.5  # ви можете змінити цей параметр

    for i in range(len(y)):
        current_date = date_array[i + sequence_length]
        
        # Кількість днів між датами
        days_diff = (last_date - current_date) / np.timedelta64(1, 'D')  
        years_diff = days_diff / 365.0
        
        weight = np.exp(-alpha * years_diff)  # e^(-0.5 * years_diff)
        sample_weights.append(weight)

    sample_weights = np.array(sample_weights)

    return X, y, scaler, sample_weights

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def create_model(sequence_length, num_features, head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, mlp_units=[128], dropout=0.3, mlp_dropout=0.4):
    inputs = Input(shape=(sequence_length, num_features))
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = Dense(dim, activation="relu")(x)
        x = Dropout(mlp_dropout)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

def predict_future(model, last_sequence, scaler, n_future=7):
    """
    Прогнозування майбутніх значень на n_future кроків уперед.
    """
    future_predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(n_future):
        current_sequence_reshaped = current_sequence.reshape((1, current_sequence.shape[0], current_sequence.shape[1]))
        next_pred = model.predict(current_sequence_reshaped, verbose=0)[0]
        future_predictions.append(next_pred)
        
        # Зсув послідовності і додавання нового передбачення
        current_sequence = np.roll(current_sequence, -1, axis=0)
        current_sequence[-1, :] = next_pred
    
    # Зворотна трансформація
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)
    
    return future_predictions

def get_model_predictions(model, X, scaler):
    """
    Отримати прогноз для наявних вибірок X (історичний проміжок)
    """
    predictions = model.predict(X, verbose=0)
    predictions = scaler.inverse_transform(predictions)
    return predictions

def visualize_predictions(df, target_currency, historical_predictions, future_predictions, last_date):
    """
    Візуалізація історичних даних та прогнозу
    """
    plt.figure(figsize=(15, 7))
    sns.set_style("whitegrid")
    plt.rcParams['font.size'] = 12
    
    # Історичні дані (синя лінія)
    plt.plot(
        df.index[-len(historical_predictions):], 
        df[target_currency].values[-len(historical_predictions):], 
        label='Історичні дані', color='blue', linewidth=2
    )
    
    # Історичні прогнозні значення (зелена лінія)
    plt.plot(
        df.index[-len(historical_predictions):], 
        historical_predictions, 
        label='Модельні значення', color='green', alpha=0.7, linewidth=2
    )
    
    # Майбутній прогноз (червона лінія)
    future_dates = pd.date_range(start=last_date, periods=len(future_predictions) + 1, freq='D')[1:]
    plt.plot(
        future_dates, 
        future_predictions, 
        label='Прогноз (коротко)', color='red', linestyle='--', linewidth=2
    )
    
    plt.axvline(x=last_date, color='gray', linestyle=':', alpha=0.5)
    
    plt.title(f'Історичні дані та прогноз курсу {target_currency}', pad=20)
    plt.xlabel('Дата', labelpad=10)
    plt.ylabel('Курс', labelpad=10)
    plt.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    return plt

def train_and_predict(df, target_currency, sequence_length=10, train_split=0.8):
    """
    Навчання моделі та отримання прогнозів (короткострокових і т.д.)
    """
    X, y, scaler, sample_weights = prepare_data(df, target_currency, sequence_length)
    
    train_size = int(len(X) * train_split)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    w_train, w_test = sample_weights[:train_size], sample_weights[train_size:]
    
    num_features = X.shape[2]
    model = create_model(sequence_length, num_features)
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        verbose=1,
        sample_weight=w_train
    )
    
    all_predictions = get_model_predictions(model, X, scaler)
    
    # Прогноз на 7 днів (як раніше)
    last_sequence = X[-1]  # остання послідовність
    future_pred_7 = predict_future(model, last_sequence, scaler, n_future=7)
    
    return model, history, all_predictions, future_pred_7, scaler, last_sequence

def main():
    # 1. Завантаження та підготовка даних
    df = pd.read_csv('data/transformed_currency_data.csv')
    df['exchangedate'] = pd.to_datetime(df['exchangedate'])
    df.set_index('exchangedate', inplace=True)
    
    target_currency = 'USD'
    
    # 2. Навчання моделі + короткостроковий прогноз (7 днів)
    model, history, historical_predictions, future_predictions_7, scaler, last_sequence = train_and_predict(df, target_currency)
    
    # 3. Візуалізація історичних + 7-денний прогноз
    plt_obj = visualize_predictions(df, target_currency, historical_predictions, future_predictions_7, df.index[-1])
    plt_obj.show()
    
    # 4. Виведення 7-денного прогнозу
    future_dates_7 = pd.date_range(start=df.index[-1], periods=len(future_predictions_7) + 1, freq='D')[1:]
    predictions_df_7 = pd.DataFrame(future_predictions_7, index=future_dates_7, columns=['Predicted Rate'])
    print("\nПрогноз курсу (7 днів):")
    print(predictions_df_7)
    
    # 5. ДОДАЄМО: Прогноз на 2025 рік (365 днів)
    n_days_2025 = 365  # кількість днів 2025 року (не враховуючи, що це не високосний рік)
    
    # Робимо прогноз, починаючи з тієї ж останньої послідовності
    future_predictions_2025 = predict_future(model, last_sequence, scaler, n_future=n_days_2025)
    
    # Формуємо діапазон дат з 2025-01-01 до 2025-12-31
    # (Якщо у вашому датасеті остання дата — 2024-12-31 чи раніше,
    # то логічно, що прогноз почнеться саме з 2025-01-01)
    dates_2025 = pd.date_range(start='2024-11-23', periods=n_days_2025, freq='D')
    
    # Записуємо в DataFrame
    predictions_2025_df = pd.DataFrame(future_predictions_2025, index=dates_2025, columns=['Predicted Rate 2025'])
    print("\nПрогноз курсу на весь 2025 рік:")
    print(predictions_2025_df)
    
    # За бажанням, можна збудувати графік чисто для 2025 року
    plt.figure(figsize=(12, 5))
    plt.plot(predictions_2025_df.index, predictions_2025_df['Predicted Rate 2025'], color='red', label='Прогноз 2025')
    plt.title('Прогноз курсу на 2025 рік')
    plt.xlabel('Дата')
    plt.ylabel('Курс')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


Epoch 1/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - loss: nan - val_loss: nan
Epoch 2/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - loss: nan - val_loss: nan
Epoch 3/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - loss: nan - val_loss: nan
Epoch 4/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - loss: nan - val_loss: nan
Epoch 5/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - loss: nan - val_loss: nan
Epoch 6/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - loss: nan - val_loss: nan
Epoch 7/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - loss: nan - val_loss: nan
Epoch 8/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - loss: nan - val_loss: nan
Epoch 9/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - loss: nan - val_loss: nan
Epoch 10/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - loss: nan - val_loss: nan
Epoch 11/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - loss: nan - val_loss: nan
Epoch 12/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: nan - val_loss: nan
Epoch 13/50
123/123 ━━━━

ValueError: non-broadcastable output operand with shape (5430,1) doesn't match the broadcast shape (5430,55)